In [57]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
#Статистика вовлеченности за день
stat=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/engagement_v30_20_11_2020.csv', \
                              sep=';',encoding='utf-8')

#список пользователей по leader-id  и email из ple
users=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/ple_user.csv', \
                              sep=';',encoding='utf-8')

#список zoom_id  мероприятий и их типов
events=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_event.csv', \
                              sep=';',encoding='utf-8')
#иногда после номера мероприятия встречается пароль - выпилим его
events.meeting_id=events.meeting_id.str.extract(r'\D*(\d*)\D*').astype('int64')

#список всех зум-сессий. В sql запросе групировка для вывода: lmp.user_name, lmp.meeting_id, lmp.email
sessions=pd.read_csv('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/zoom_sesion.csv', \
                              sep=';',encoding='utf-8')
sessions.meeting_id=sessions.meeting_id.astype('int64')

In [67]:
# Преодразуем к единому файлу со всей инфой
# Сшиваем по полю meeting_id
sessions_merged = pd.merge(
    sessions,
    events,
    on=['meeting_id'],
    how='left' ).fillna(0)

# Сшиваем  c users по полю email
sessions_merged = pd.merge(
    sessions_merged,
    users,
    on=['email'],
    how='left' ).fillna(0)

sessions_merged.leaderID=sessions_merged.leaderID.astype('int')
sessions_merged['zoom_name_leaderID']= sessions_merged.user_name.str.extract(r'\D*(\d*)\D*')
sessions_merged['zoom_name_leaderID']= sessions_merged['zoom_name_leaderID'].str.lstrip('0')
sessions_merged['zoom_name_leaderID']= pd.to_numeric(sessions_merged['zoom_name_leaderID'],downcast='integer'). \
                                        fillna(0).astype('int')

sessions_merged=sessions_merged.rename(columns={ 'leaderID': 'email_leaderID'})

#Логика текущая такая: для каждого сеанса
#1. если есть в имени номер - используем его
#2. если в имени номера нет - берем leader_id
#Таким образом если с разными email но одним именем - то leaderID будет из имени браться и только при его отсутствии - из email

condlist = [sessions_merged.zoom_name_leaderID==0, sessions_merged.zoom_name_leaderID>0]
choicelist = [sessions_merged.email_leaderID, sessions_merged.zoom_name_leaderID]
sessions_merged['leaderID']= np.select(condlist,choicelist)

#Теперь делаем преобразование
#1. Отбираем тех, кто присутствовал больше 10 минут -  time_on>600
#2. Группируем по: leaderID, event_type, День
#Считаем количество мероприятий каждого типа
#

session_by_type = sessions_merged[sessions_merged.time_on>600].\
    groupby(['leaderID','День','event_type']). \
    agg(zoom_count=('meeting_id', 'count')).fillna(0)
session_by_type=session_by_type.reset_index()

#Теперь все типы Зумов сделаем колонками
session_by_day = pd.pivot_table(session_by_type,index=['leaderID','День'],columns='event_type', values='zoom_count'). \
                reset_index().fillna(0)
#Удалим не типированные Зумы
session_by_day=session_by_day.drop(columns=[0])

# Сшиваем статистику с Зумами. Сшивка по полям leaderID и День
stat_merged = pd.merge(
    stat,
    session_by_day,
    on=['leaderID','День'],
    how='left' ).fillna(0)

#Переименуем в понятные всем имена - чтобы записи на мероприятия с Зумами не путались
stat_merged = stat_merged.rename(columns={ 'Акселерационная программа': 'Zoom по Акселерационной программе',
                         'Визионерская лекция':'Zoom по Визионерским лекциям',
                         'Лаборатория':'Zoom по Лабораториям',
                         'Лекция':'Zoom по Лекциям',
                         'Мастер-класс':'Zoom по Мастер-классам',
                         'Общение с экспертами':'Zoom по Общению с экспертами',
                         'Повышение эффективности': 'Zoom по Повышению эффективности'
                        })

In [68]:
# Пока нет ограничений на максимум записей 
min_rep= pd.DataFrame()
min_rep['leaderID']=stat_merged.leaderID
min_rep['День']=stat_merged['День']
min_rep['Образовательная вовлеченность']= stat_merged['Запись на МК'].apply(lambda x: 1 if x>0  else 0)*0.1 + \
        stat_merged['Запись на Лабораторию'].apply(lambda x: 1 if x>0  else 0)*0.1 + \
        stat_merged['Запись на повышение эффективности'].apply(lambda x: 1 if x>0  else 0)*0.1  + \
        stat_merged['Оставлен Фидбек на мероприятие (хотя бы один ответ)']*0.1 + \
        stat_merged['Создание Темы изучения в Ком.профиле']*1 + \
        stat_merged['Zoom по Визионерским лекциям'].apply(lambda x: 1 if x>0  else 0) *2 +\
        stat_merged['Zoom по Лабораториям'].apply(lambda x: 1 if x>0  else 0)*1 +  \
        stat_merged['Zoom по Мастер-классам'].apply(lambda x: 1 if x>0  else 0)*1 +\
        stat_merged['Zoom по Лекциям'].apply(lambda x: 1 if x>0  else 0)*1 + \
        stat_merged['Zoom по Повышению эффективности'].apply(lambda x: 1 if x>0  else 0)*1

min_rep['Проектная вовлеченность']= stat_merged['Запись на Акслератор'].apply(lambda x: 1 if x>0  else 0)*0.1 + \
        stat_merged['Запись на встречу с экспертом']*0.1+\
        stat_merged['Загружена в PT презентация по проекту']*1 + stat_merged['Проект в ПТ размечен']*1 + \
        stat_merged['Участником создана задача в Командном профиле']*1 + stat_merged['Выполнена задача в ком.профиле']*2+ \
        stat_merged['Запись на Питч-сессию']*0.1 + \
        stat_merged['Zoom по Акселерационной программе'].apply(lambda x: 1 if x>0  else 0)*2+ \
        stat_merged['Zoom по Общению с экспертами']*1 + stat_merged['Питч-сессии']*1

min_rep['Социальная вовлеченность']= stat_merged['Оставлен Фидбек на мероприятие (хотя бы один ответ)']*1 +\
        stat_merged['Запись на встречу с экспертом']*0.1+ \
        stat_merged['Создан Продукт в Edumap']*2 +\
        stat_merged['Поставлен лайк проекту в ПТ']*1 + stat_merged['Поставлен лайк продукту (карточке) в Edumap']*1 +\
        stat_merged['Запись на Питч-сессию']*0.1 + stat_merged['Создано вакансий']*2 +  stat_merged['Откликов на вакансии']*1 +\
        stat_merged['Zoom по Общению с экспертами']*1 + stat_merged['Питч-сессии']*2

min_rep['Суммарная итоговая вовлеченность']=min_rep['Образовательная вовлеченность']+min_rep['Проектная вовлеченность']+ \
         min_rep['Социальная вовлеченность']   

min_rep['Призрачный пользователь'] = min_rep['Суммарная итоговая вовлеченность'].apply(lambda x: 1 if x>0  else 0) 

min_rep['Живой пользователь'] = min_rep['Суммарная итоговая вовлеченность'].apply(lambda x: 1 if x>=1 else 0) 

min_rep['Вовлеченный пользователь'] = min_rep['Суммарная итоговая вовлеченность'].apply(lambda x: 1 if x>1 else 0) 

min_rep['Активный пользователь'] = min_rep['Суммарная итоговая вовлеченность'].apply(lambda x: 1 if x>2 else 0) 

min_rep['Оплативший участие'] = stat_merged['Оплативший участие']

min_rep['Команда'] = stat_merged['Название проекта']


min_rep.head(3)

,leaderID,День,Образовательная вовлеченность,Проектная вовлеченность,Социальная вовлеченность,Суммарная итоговая вовлеченность,Призрачный пользователь,Живой пользователь,Вовлеченный пользователь,Активный пользователь,Оплативший участие,Команда
0,56,2020-11-07,2.4,0.0,2.0,4.4,1,1,1,1,0,0
1,56,2020-11-08,2.4,0.0,3.0,5.4,1,1,1,1,0,0
2,56,2020-11-09,2.4,0.0,2.0,4.4,1,1,1,1,0,0


In [69]:
#Отчет по всем, часть про вовлеченности
rep_by_day = min_rep[min_rep['Живой пользователь']==1].groupby(['День']). \
    agg(edu_mean     = ('Образовательная вовлеченность', 'mean'), \
        project_mean = ('Проектная вовлеченность', 'mean'),\
        social_mean  = ('Социальная вовлеченность', 'mean'), \
        result_mean  = ('Суммарная итоговая вовлеченность','mean')).reset_index().fillna(0)

rep_by_day['Средняя по типу вовлеченность'] = (rep_by_day.edu_mean + rep_by_day.project_mean + rep_by_day.social_mean)/3

rep_by_day = rep_by_day.rename(columns={'edu_mean'    : 'Средняя Образовательная вовлеченность живых',
                                        'project_mean': 'Средняя Проектная вовлеченность живых',
                                        'social_mean' : 'Средняя Социальная вовлеченность живых',
                                        'result_mean' : 'Средняя Суммарная  вовлеченность живых'
                                       })
#Отчет по всем, часть про статусы
rep_by_type = min_rep.groupby(['День']). \
    agg(ghost_sum    = ('Призрачный пользователь','sum'), \
        live_sum     = ('Живой пользователь','sum'),\
        engaged_sum  = ('Вовлеченный пользователь','sum'),\
        active_sum   = ('Активный пользователь','sum')  ).reset_index().fillna(0)

rep_by_type = rep_by_type.rename(columns={  'ghost_sum'   : 'Число Призрачных пользователей',
                                            'live_sum'    : 'Число Живых пользователей',
                                            'engaged_sum' : 'Число Вовлеченных пользователей',
                                            'active_sum'  : 'Число Активных пользователей'
                                       })

In [70]:
#Отчет по оплаченным, часть про вовлеченности
rep_payed_day = min_rep[(min_rep['Живой пользователь'] == 1)& (min_rep['Оплативший участие'] == 1)].groupby(['День']). \
    agg(edu_mean     = ('Образовательная вовлеченность', 'mean'), \
        project_mean = ('Проектная вовлеченность', 'mean'),\
        social_mean  = ('Социальная вовлеченность', 'mean'), \
        result_mean  = ('Суммарная итоговая вовлеченность','mean')).reset_index().fillna(0)

rep_payed_day['Средняя по типу вовлеченность'] = (rep_payed_day.edu_mean + rep_payed_day.project_mean + \
                                                  rep_payed_day.social_mean)/3

rep_payed_day = rep_payed_day.rename(columns={'edu_mean'    : 'Средняя Образовательная вовлеченность живых',
                                        'project_mean': 'Средняя Проектная вовлеченность живых',
                                        'social_mean' : 'Средняя Социальная вовлеченность живых',
                                        'result_mean' : 'Средняя Суммарная  вовлеченность живых'
                                       })
#Отчет по оплаченным, часть про статусы
rep_payed_type = min_rep[min_rep['Оплативший участие'] == 1].groupby(['День']). \
    agg(ghost_sum    = ('Призрачный пользователь','sum'), \
        live_sum     = ('Живой пользователь','sum'),\
        engaged_sum  = ('Вовлеченный пользователь','sum'),\
        active_sum   = ('Активный пользователь','sum')  ).reset_index().fillna(0)

rep_payed_type = rep_payed_type.rename(columns={  'ghost_sum'   : 'Число Призрачных пользователей',
                                            'live_sum'    : 'Число Живых пользователей',
                                            'engaged_sum' : 'Число Вовлеченных пользователей',
                                            'active_sum'  : 'Число Активных пользователей'
                                       })

In [71]:
team_by_day_live = min_rep[min_rep['Живой пользователь']==1].groupby(['День','Команда']). \
    agg( result_mean_live  = ('Суммарная итоговая вовлеченность','mean')).\
    reset_index().fillna(0)

team_by_day_all = min_rep.groupby(['День','Команда']). \
    agg( result_mean_all  = ('Суммарная итоговая вовлеченность','mean')).\
    reset_index().fillna(0)

#Команда '0' - это все без команд, поэтому удалим эту псевдокоманду из данных
team_by_day_live = team_by_day_live.loc[team_by_day_live['Команда']!='0']
team_by_day_all = team_by_day_all.loc[team_by_day_all['Команда']!='0']

team_by_day_all['Мертвая команда (по всем)'] = team_by_day_all['result_mean_all'].apply(lambda x: 1 if x==0 else 0) 
team_by_day_all['Живая команда (по всем)'] = team_by_day_all['result_mean_all'].apply(lambda x: 1 if x>1 else 0) 
team_by_day_all['Вовлеченная команда (по всем)'] = team_by_day_all['result_mean_all'].apply(lambda x: 1 if x>2 else 0) 
team_by_day_all['Активная команда (по всем)'] = team_by_day_all['result_mean_all'].apply(lambda x: 1 if x>3 else 0) 

team_by_day_live['Мертвая команда (по живым)'] = team_by_day_all['Мертвая команда (по всем)'] 
team_by_day_live['Живая команда (по живым)'] = team_by_day_live['result_mean_live'].apply(lambda x: 1 if x>1 else 0) 
team_by_day_live['Вовлеченная команда (по живым)'] = team_by_day_live['result_mean_live'].apply(lambda x: 1 if x>2 else 0) 
team_by_day_live['Активная команда (по живым)'] = team_by_day_live['result_mean_live'].apply(lambda x: 1 if x>3 else 0) 

#Теперь просуммируем по дням

rep_team_live = team_by_day_live.groupby(['День']).agg(  dead = ('Мертвая команда (по живым)', 'sum'),
                                                         live = ('Живая команда (по живым)','sum'),
                                                         eng  = ('Вовлеченная команда (по живым)','sum'),
                                                         activ= ('Активная команда (по живым)','sum')).reset_index().fillna(0)
rep_team_live = rep_team_live.rename(columns={'dead' : 'Мертвых команд',
                                              'live' : 'Живых команд',
                                              'eng'  : 'Вовлеченных команд',
                                              'activ': 'Активных команд'
                                       })


rep_team_all = team_by_day_all.groupby(['День']).agg( dead = ('Мертвая команда (по всем)', 'sum'),
                                                      live = ('Живая команда (по всем)','sum'),
                                                      eng  = ('Вовлеченная команда (по всем)','sum'),
                                                      activ= ('Активная команда (по всем)','sum')).reset_index().fillna(0)
rep_team_all = rep_team_all.rename(columns={'dead' : 'Мертвых команд',
                                              'live' : 'Живых команд',
                                              'eng'  : 'Вовлеченных команд',
                                              'activ': 'Активных команд'
                                       })

In [73]:
with pd.ExcelWriter('C:/Users/d.muravlyansky/Documents/UNTI/Архипелаг/Report_20_11_2020.xlsx') as writer:  
    rep_team_all.to_excel(writer, sheet_name='Команды по всем участникам',index=False)
    rep_team_live.to_excel(writer, sheet_name='Команды по живым',index=False)
    rep_by_day.to_excel(writer, sheet_name='Вовлеченность всех',index=False)
    rep_by_type.to_excel(writer, sheet_name='Статусы всех',index=False)
    rep_payed_day.to_excel(writer, sheet_name='Вовлеченность оплаченных',index=False)
    rep_payed_type.to_excel(writer, sheet_name='Статусы оплаченных',index=False)

In [48]:
pd.set_option('display.max_columns', 35) 
min_rep[(min_rep['Живой пользователь'] == 1)& (min_rep['Оплативший участие'] == 1)].head(3)

,leaderID,День,Загружена в PT презентация по проекту,Проект в ПТ размечен,Участником создана задача в Командном профиле,Выполнена задача в ком.профиле,Верифицировано решение в ком.профиле,Создание Темы изучения в Ком.профиле,Закрыта Тема изучения в Ком.профиле,Запись на Акслератор,Запись на встречу с экспертом,Запись на Лабораторию,Запись на МК,Запись на повышение эффективности,Запись на Питч-сессию,Отредактирована цель,Оценено продвижение к цели в ЛК,Оставлен Фидбек на мероприятие (хотя бы один ответ),Создан Продукт в Edumap,Поставлен лайк проекту в ПТ,Поставлен лайк продукту (карточке) в Edumap,Создано вакансий,Откликов на вакансии,Оплативший участие,Название проекта,Zoom по Акселерационной программе,Zoom по Визионерским лекциям,Zoom по Лабораториям,Zoom по Лекциям,Zoom по Мастер-классам,Zoom по Общению с экспертами,Питч-сессии,Zoom по Повышению эффективности
15,69,2020-11-07,1,1,0,0,0,0,0,2,0,1,1,0,0,0,0,1,0,1,0,2,0,1,Эмоциональный помощник презентанта с использов...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
